# Nettoyage des données

## 0. Set up

### Mise en place de l'environnement

In [1]:
#Importation des Modules nécessaires
import pandas as pd
import sqlalchemy
import mysql.connector

#Pour que python reconnaisse un dossier comme un module il faut ajouter en son sein un fichier __init__.py 
#et également l'ajouter au PATH

import sys
sys.path.insert(0, "/home/apprenant/PycharmProjects/American_dream")


#On peut ensuite importer le dossier comme un module

from src.d00_utils.mysql_utils import mysql_connect, save_to_mysql
connect = mysql_connect()

### Importation des données depuis my sql

In [2]:
df1 = pd.read_sql("survey_1",con=connect)
dfk = pd.read_sql("survey_k",con=connect)

## 1. Selection des colonnes

J'affiche un échantillno de mes données pour analyser la pertinence des différentes colonnes

In [3]:
print(df1.head())
print(df1.columns)

   index  Survey Year           Timestamp  SalaryUSD        Country  \
0      0         2020 2020-01-04 18:50:34   115000.0  United States   
1      1         2020 2020-01-04 10:43:02   100000.0  United States   
2      2         2020 2020-01-04 09:51:46   100000.0          Spain   
3      3         2020 2020-01-04 01:08:54    70000.0  United States   
4      4         2020 2020-01-03 15:28:54   110000.0  United States   

  PostalCode       PrimaryDatabase  YearsWithThisDatabase  \
0         03  Microsoft SQL Server                     15   
1       None                 Other                      6   
2      28046  Microsoft SQL Server                      2   
3      94133  Microsoft SQL Server                      3   
4      95354                Oracle                     30   

                                      OtherDatabases    EmploymentStatus  ...  \
0  Microsoft SQL Server, MongoDB, Azure SQL DB (a...  Full time employee  ...   
1                                      MySQL

Je choisis uniquement les colonnes qui pourraient m'être utiles pour mon étude

In [4]:
df1 = df1[['index','SalaryUSD','Country','PostalCode','EmploymentStatus','JobTitle','ManageStaff','YearsWithThisTypeOfJob',
          'HowManyCompanies','OtherPeopleOnYourTeam', 'CompanyEmployeesOverall',
          'PopulationOfLargestCityWithin20Miles','EmploymentSector', 'LookingForAnotherJob', 'CareerPlansThisYear',
          'Gender', 'OtherJobDuties']]

Je fais la même chose pour la seconde table

## 2. Valeurs manquantes

### Recherche des valeurs manquantes 

In [5]:
print(df1.isnull().sum()) 
print(df1.shape)

index                                      0
SalaryUSD                                  0
Country                                    0
PostalCode                              1390
EmploymentStatus                           0
JobTitle                                   0
ManageStaff                                0
YearsWithThisTypeOfJob                     0
HowManyCompanies                           0
OtherPeopleOnYourTeam                      0
CompanyEmployeesOverall                    0
PopulationOfLargestCityWithin20Miles       0
EmploymentSector                           0
LookingForAnotherJob                       0
CareerPlansThisYear                        0
Gender                                     0
OtherJobDuties                           620
dtype: int64
(8627, 17)


Il me manque 1390 veleurs pour PostalCode et 620 pour OtherJobDUties sur un total de 8627 lignes.
Ces deux colonnes ne sont pas au centre de mon études donc ça se vaut pas le cout de supprimer toute la ligne pour elles

### traitement de Postal code

Cela ne ferait pas de sens de remplacer le code postal par un code postal arbitraire

### Traitement de OtherJobDuties

In [6]:
print(df1.OtherJobDuties.unique())
print(df1.OtherJobDuties.value_counts())

['Not Asked'
 'DBA (Development Focus - tunes queries, indexes, does deployments), Developer: App code (C#, JS, etc), Developer: T-SQL'
 None
 'DBA (Development Focus - tunes queries, indexes, does deployments), Developer: T-SQL'
 'DBA (Development Focus - tunes queries, indexes, does deployments), Developer: App code (C#, JS, etc), Developer: Business Intelligence (SSRS, PowerBI, etc), Developer: T-SQL'
 'Architect, DBA (General - splits time evenly between writing & tuning queries AND building & troubleshooting servers), Developer: Business Intelligence (SSRS, PowerBI, etc), Developer: T-SQL'
 'DBA (Production Focus - build & troubleshoot servers, HA/DR)' 'Engineer'
 'DBA (Development Focus - tunes queries, indexes, does deployments)'
 'Developer: T-SQL'
 'Architect, Developer: Business Intelligence (SSRS, PowerBI, etc), Developer: T-SQL'
 'DBA (Development Focus - tunes queries, indexes, does deployments), DBA (General - splits time evenly between writing & tuning queries AND buildi

Comme il y a trop de valeur unique différente pour OtherJobDuties, je  ne peux remplacer les valeurs manquantes par une valeur standard

### Conclusion sur les valeurs manquantes

Je choisis donc de conserver l'ensemble des données dans ma base mais je note un point de vigilance quand j'aurais à utiliser les variables codePostal et OtherJobDuties

### Même travail pour la seconde base de données

## 3. Traitement des dates

In [7]:
print(df1.dtypes)

index                                     int64
SalaryUSD                               float64
Country                                  object
PostalCode                               object
EmploymentStatus                         object
JobTitle                                 object
ManageStaff                              object
YearsWithThisTypeOfJob                    int64
HowManyCompanies                         object
OtherPeopleOnYourTeam                    object
CompanyEmployeesOverall                  object
PopulationOfLargestCityWithin20Miles     object
EmploymentSector                         object
LookingForAnotherJob                     object
CareerPlansThisYear                      object
Gender                                   object
OtherJobDuties                           object
dtype: object


Aucune de mes colonnes ne correspond à une date, je n'ai donc pas à faire de traitement

## 4. Traitement des doublons

In [8]:
print(df1.duplicated().value_counts())

False    8627
dtype: int64


Aucune des colonnes de ma base n'est dupliquée, je vérifie cependant que c'est également le cas sur la colonne index

In [9]:
print(df1.duplicated('index').value_counts())

False    8627
dtype: int64


On est bon.

## to be continued

### Avant de partir je sauvegarde mon travail dans my sql

In [10]:
save_to_mysql(db_connect=connect,df_to_save=df1,df_name='survey_1_v1.0')